# Shortest Path Discovery using Q Learning

Import libraries

In [1]:
import numpy as np

In [2]:
# Mendefinisikan bentuk environment (yaitu, states-nya)
environment_rows = 11
environment_columns = 11

# Buat array numpy 3D untuk menyimpan nilai Q saat ini untuk setiap pasangan state dan action: Q(s, a)
# Array berisi 11 baris dan 11 kolom (untuk menyesuaikan dengan bentuk environment), serta third "action" dimension.
# Action dimension terdiri dari 4 lapisan yang memungkinkan kita untuk melacak nilai Q untuk setiap aksi yang mungkin terjadi.
# Setiap state (lihat sel berikutnya untuk deskripsi tindakan yang mungkin dilakukan).
# Nilai dari setiap pasangan (state, action) diinisialisasi ke 0.
q_values = np.zeros((environment_rows, environment_columns, 4))

In [3]:
# Mendefinisikan actions
# Kode numerik action: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

In [4]:
# Buat array numpy 2D untuk menyimpan reward untuk setiap state.
# Array berisi 11 baris dan 11 kolom (agar sesuai dengan bentuk environment),
# Dan setiap nilai diinisialisasi ke -100.
rewards = np.full((environment_rows, environment_columns), -100.)
rewards[0, 5] = 100. #set the reward for the packaging area (i.e., the goal) to 100

# Mendefinisikan lokasi lorong untuk baris 1 sampai 9
aisles = {}
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(11)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(11)]

# Set reward tiap lorong
for row_index in range(1, 10):
  for column_index in aisles[row_index]:
    rewards[row_index, column_index] = -1.

# print rewards matrix
for row in rewards:
  print(row)

[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100. -100. -100. -100. -100. -100. -100.]


In [5]:
# Mendefinisikan fungsi yang menentukan apakah lokasi yang ditentukan adalah status terminal
def is_terminal_state(current_row_index, current_column_index):
  # jika reward untuk lokasi ini adalah -1, maka lokasi ini bukan merupakan terminal state
  if rewards[current_row_index, current_column_index] == -1.:
    return False
  else:
    return True

# Mendefinisikan fungsi yang akan memilih lokasi awal yang acak dan bukan terminal
def get_starting_location():
  # Menetapkan indeks baris dan kolom secara acak
  current_row_index = np.random.randint(environment_rows)
  current_column_index = np.random.randint(environment_columns)
  # Lanjutkan memilih indeks baris dan kolom secara acak hingga keadaan non-terminal teridentifikasi
  while is_terminal_state(current_row_index, current_column_index):
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
  return current_row_index, current_column_index

# Mendefinisikan algoritme epsilon greedy yang akan memilih tindakan mana yang akan diambil selanjutnya (yaitu, ke mana harus bergerak selanjutnya)
def get_next_action(current_row_index, current_column_index, epsilon):
  # Jika nilai yang dipilih secara acak antara 0 dan 1 kurang dari epsilon,
  # Maka pilihlah nilai yang paling layak dari tabel Q untuk keadaan ini.
  if np.random.random() < epsilon:
    return np.argmax(q_values[current_row_index, current_column_index])
  else: # Memilih random action
    return np.random.randint(4)

# Mendefinisikan fungsi yang akan mendapatkan lokasi berikutnya berdasarkan action yang dipilih
def get_next_location(current_row_index, current_column_index, action_index):
  new_row_index = current_row_index
  new_column_index = current_column_index
  if actions[action_index] == 'up' and current_row_index > 0:
    new_row_index -= 1
  elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
    new_column_index += 1
  elif actions[action_index] == 'down' and current_row_index < environment_rows - 1:
    new_row_index += 1
  elif actions[action_index] == 'left' and current_column_index > 0:
    new_column_index -= 1
  return new_row_index, new_column_index

# Tentukan fungsi yang akan mendapatkan jalur terpendek antara lokasi mana pun di dalam gudang yang yang boleh dilalui robot dan lokasi pengemasan barang.
def get_shortest_path(start_row_index, start_column_index):
  # Segera kembali jika lokasi awal tidak valid
  if is_terminal_state(start_row_index, start_column_index):
    return []
  else: # Jika lokasi ini merupakan lokasi awal yang 'legal'
    current_row_index, current_column_index = start_row_index, start_column_index
    shortest_path = []
    shortest_path.append([current_row_index, current_column_index])
    # Terus bergerak di sepanjang jalan sampai kita mencapai tujuan (yaitu, lokasi pengemasan barang)
    while not is_terminal_state(current_row_index, current_column_index):
      # Mendapatkan action terbaik untuk diambil
      action_index = get_next_action(current_row_index, current_column_index, 1.)
      # Pindah ke lokasi berikutnya di jalur, dan tambahkan lokasi baru ke daftar
      current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
      shortest_path.append([current_row_index, current_column_index])
    return shortest_path

In [6]:
# Mendifinisikan parameter training
epsilon = 0.9 # Persentase waktu ketika kita harus mengambil tindakan terbaik (bukan tindakan acak)
discount_factor = 0.9 # Faktor diskon untuk hadiah di masa mendatang
learning_rate = 0.9 # Tingkat di mana agen AI harus belajar

# Jalankan melalui 1000 episode pelatihan
for episode in range(1000):
  # Dapatkan lokasi awal untuk episode ini
  row_index, column_index = get_starting_location()

  # Lanjutkan pengambilan action (yaitu, bergerak) sampai kita mencapai keadaan terminal
  # (yaitu, hingga kita mencapai area pengemasan barang atau menabrak lokasi penyimpanan barang)
  while not is_terminal_state(row_index, column_index):
    # Memilih action yang akan diambil (misalnya, ke mana harus bergerak selanjutnya)
    action_index = get_next_action(row_index, column_index, epsilon)

    # Melakukan action yang dipilih, dan beralih ke state berikutnya (misalnya, berpindah ke lokasi berikutnya)
    old_row_index, old_column_index = row_index, column_index # Menyimpan indeks baris dan kolom yang lama
    row_index, column_index = get_next_location(row_index, column_index, action_index)

    # Menerima reward karena telah berpindah ke state yang baru, dan menghitung temporal difference
    reward = rewards[row_index, column_index]
    old_q_value = q_values[old_row_index, old_column_index, action_index]
    temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

    # Perbarui nilai Q untuk pasangan state dan action sebelumnya
    new_q_value = old_q_value + (learning_rate * temporal_difference)
    q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

Training complete!


In [7]:
# Menampilkan beberapa jalur terpendek
print(get_shortest_path(3, 9)) # Dimulai pada baris 3, kolom 9
print(get_shortest_path(5, 0)) # Dimulai pada baris 5, kolom 0
print(get_shortest_path(9, 5)) # Dimulai pada baris 9, kolom 5

[[3, 9], [2, 9], [1, 9], [1, 8], [1, 7], [1, 6], [1, 5], [0, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [4, 3], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]
[[9, 5], [9, 6], [9, 7], [8, 7], [7, 7], [7, 6], [7, 5], [6, 5], [5, 5], [5, 6], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]


In [8]:
# Menampilkan contoh jalur terpendek yang dibalik
path = get_shortest_path(5, 2) # Pergi ke baris 5, kolom 3
path.reverse()
print(path)

[[0, 5], [1, 5], [1, 6], [1, 7], [2, 7], [3, 7], [3, 6], [3, 5], [3, 4], [3, 3], [4, 3], [5, 3], [5, 2]]
